In [1]:
#!/usr/bin/env python
# coding: utf-8
import json
import pandas as pd    
from requests import get
from bs4 import BeautifulSoup

In [2]:
def extract_html(url):
    """
    Function for extracting html content using BeautifulSoup
    """
    response = get(url, allow_redirects=False)
    html = response.text
    soup = BeautifulSoup(html, "lxml-xml")
    return(soup)

In [3]:
def add_to_json(data, title, summary, url, category):
    """
    Function for adding data contents to a json file
    """
    data['Article'].append({
        'Title': title,
        'Content': summary,
        'Link': url,
        'Category': category
    })
    return(data['Article'])

In [4]:
def save_json(data, fname):
    """
    Function for saving json file
    """
    with open('{}.json'.format(fname), 'w') as outfile:
        json.dump(data, outfile)

In [5]:
def get_pages_num(soup):
    """
    Function for retrieving the number of pages to be scrapped for each category
    """
    num = int(soup.find("div", {"class": "pagination"}).findAll("a")[-2].text)
    return num

In [6]:
data = {}
data['Article'] = []

soup = extract_html("https://almashareq.com/ar/archives")
stories = soup.findAll("div", {"class": "article--listing"})

for n in range(3, len(stories)):
    soup = extract_html("https://almashareq.com"+stories[n].find("div", {"class": "article--listing__text"}).find("a").attrs['href'])
    
    for m in range(get_pages_num(soup)):
        #Access each category page
        soup1 = extract_html("https://almashareq.com"+stories[n].find("div", {"class": "article--listing__text"}).find("a").attrs['href']+"?page={}".format(m))
        sub_stories = soup1.findAll("div", {"class": "article--block"})
        
        for k in range(len(sub_stories)):
            title = sub_stories[k].find('h5', {"class": "article__title"}).text
            url = "https://almashareq.com"+sub_stories[k].find('a').attrs['href']
            category = sub_stories[k].find('a', {"class", "article__keyword"}).text
            soup2 = extract_html("https://almashareq.com"+sub_stories[k].find('a').attrs['href'])
            divs = soup2.find("div", {"class": "article__content"})
            
            if divs != []:
                ps = divs.findAll('p', {"class": "article__p"})
                content = ""
                for p in ps:
                    content += p.text
                add_to_json(data, title, content, url, category)
        
save_json(data, 'results')

In [15]:
df = pd.json_normalize(data['Article'])
df.to_csv("Datasets/Arabic_News_AL-Mashareq.csv", index=False)
df

,Title,Content,Link,Category
0,\n المقاتلون السريان يثبتون جدارتهم ف...,\nيروي القياديان العسكريان من الطائفة السرياني...,https://almashareq.com/ar/articles/cnmi_am/fea...,أمن
1,\n اليمن ينظم أموال الزكاة\n,\nأكد مسؤولون في وزارة الادارة المحلية اليمنية...,https://almashareq.com/ar/articles/cnmi_am/fea...,أمن
2,\n لبنان يجدد المعركة ضد الإرهاب بعد ...,\nتعهدت السلطات اللبنانية بمواصلة الحرب ضد الإ...,https://almashareq.com/ar/articles/cnmi_am/fea...,أمن
3,\n اليمن يطبق خطة أمنية مشتركة في حضر...,\nتنفذ قوات الأمن اليمنية المشتركة خطة موحدة ل...,https://almashareq.com/ar/articles/cnmi_am/fea...,أمن
4,\n صنعاء تشدد التدابير الأمنية خلال ر...,\nشددت وزارة الداخلية اليمنية التدابير الأمنية...,https://almashareq.com/ar/articles/cnmi_am/fea...,أمن
...,...,...,...,...
3595,\n القاعدة تشجب تشكيك بعض المسلمين بد...,\nضمن جهوده الدؤوبة لإثبات مصداقيته أمام الرأي...,https://almashareq.com/ar/articles/cnmi_am/fea...,تحليل
3596,\n انقسامات الحوثيين تدفع زعماء يمنيي...,\nتشهد العاصمة صنعاء هروبا متواصلا لأعضاء مجلس...,https://almashareq.com/ar/articles/cnmi_am/fea...,تحليل
3597,\n هل داعش هي من صُنع الغرب؟\n,\nتسود العالم الإسلامي نظرية راسخة مآلها أن تن...,https://almashareq.com/ar/articles/cnmi_am/fea...,تحليل
3598,\n هل داعش هي من صُنع الغرب؟\n,\nتسود العالم الإسلامي نظرية راسخة مآلها أن تن...,https://almashareq.com/ar/articles/cnmi_am/fea...,تحليل
